In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dspy

if hasattr(dspy, 'cache') and hasattr(dspy.cache, 'disk_cache'):
    dspy.cache.disk_cache.clear()
    print("clear")

clear


In [3]:
class ChatSignature(dspy.Signature):
    """Respond to user in a conversation"""
    conversation_history: str = dspy.InputField(desc="Previous messages")
    user_message: str = dspy.InputField(desc="Current user input")
    response: str = dspy.OutputField(desc="Assistant's response")

class Chatbot:
    def __init__(self):
        self.predictor = dspy.Predict(ChatSignature)
        self.history = []
    
    def chat(self, user_message: str) -> str:
        # Format history
        history_str = "\n".join([f"{m['role']}: {m['content']}" for m in self.history])
        
        # Get response using signature
        result = self.predictor(conversation_history=history_str, user_message=user_message)
        
        # Update history
        self.history.append({"role": "user", "content": user_message})
        self.history.append({"role": "assistant", "content": result.response})
        
        return result.response


In [4]:
from package.base import DriverLM, ModelResponse, Usage
import httpx
from typing import Any

ollama_client = httpx.Client(timeout=600.0)

def ollama_request_fn(messages:list[dict[str, Any]], temperature:float=0.0, **kwargs)->dict:
    base_url = 'http://localhost:11434/api/chat'
    # print(messages)
    response = ollama_client.post(
        base_url,
        json={
            "model": "llama3.2-vision:11b",
            "messages": messages,
            "stream": False,
            "options": {"temperature": temperature}
        }
    )
    response.raise_for_status()
    return response.json()  # Return full Ollama response

def ollama_output_fn(response:dict)->ModelResponse:
    content = response.get("message", {}).get("content", "")
    model = response.get("model", "custom")
    
    usage = Usage(
        prompt_tokens=response.get("prompt_eval_count", 0),
        completion_tokens=response.get("eval_count", 0),
        total_tokens=response.get("prompt_eval_count", 0) + response.get("eval_count", 0)
    )
    
    return ModelResponse.from_text(text=content.strip(), usage=usage, model=model)

In [5]:
response = ollama_request_fn(messages=[{"role": "user", "content": "Hi"}])
ollama_output_fn(response)

ModelResponse(id='chatcmpl-77d3b091-9730-4d3e-9abb-b2e6af72a927', created=1768742690, model='llama3.2-vision:11b', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='How can I assist you?', role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None, reasoning_content=None))], usage={'prompt_tokens': 10, 'completion_tokens': 7, 'total_tokens': 17}, cache_hit=False)

In [6]:
import boto3
from typing import Any
from package.base import DriverLM, ModelResponse, Usage


def bedrock_request_fn(messages: list[dict[str, Any]], temperature: float = 0.0, **kwargs) -> dict:
    """
    Request function for AWS Bedrock Converse API.
    
    DSPy passes messages in OpenAI format:
    [
        {"role": "system", "content": "..."},
        {"role": "user", "content": "..."},
        {"role": "assistant", "content": "..."},
        ...
    ]
    
    Bedrock Converse API expects:
    - system: list of system messages (separate parameter)
    - messages: list of user/assistant turns (no system role)
    """
    
    client = boto3.client('bedrock-runtime', region_name='us-east-1')
    
    # Separate system messages from conversation
    system_messages = []
    conversation_messages = []
    
    for msg in messages:
        if msg["role"] == "system":
            system_messages.append({"text": msg["content"]})
        else:
            conversation_messages.append({
                "role": msg["role"],
                "content": [{"text": msg["content"]}]
            })
    
    # Build request
    request_params = {
        "modelId": "us.amazon.nova-lite-v1:0",
        "messages": conversation_messages,
        "inferenceConfig": {
            "temperature": temperature,
            "maxTokens": kwargs.get("max_tokens", 2048),
        }
    }
    
    # Add system messages if present
    if system_messages:
        request_params["system"] = system_messages
    
    # Call Bedrock
    response = client.converse(**request_params)
    
    return response


def bedrock_output_fn(response: dict) -> ModelResponse:
    """
    Parse Bedrock Converse API response into ModelResponse.
    
    Bedrock response format:
    {
        "output": {
            "message": {
                "role": "assistant",
                "content": [{"text": "..."}]
            }
        },
        "usage": {
            "inputTokens": 100,
            "outputTokens": 50,
            "totalTokens": 150
        },
        "stopReason": "end_turn",
        "metrics": {...}
    }
    """
    
    # Extract content
    content = response["output"]["message"]["content"][0]["text"]
    
    # Extract usage
    usage_data = response.get("usage", {})
    usage = Usage(
        prompt_tokens=usage_data.get("inputTokens", 0),
        completion_tokens=usage_data.get("outputTokens", 0),
        total_tokens=usage_data.get("totalTokens", 0)
    )
    
    # Get model ID from response metadata
    model = response.get("ResponseMetadata", {}).get("HTTPHeaders", {}).get("x-amzn-bedrock-model-id", "bedrock-model")
    
    return ModelResponse.from_text(text=content, usage=usage, model=model)



In [7]:
response = bedrock_request_fn(messages=[{"role": "user", "content": "Hi"}])
bedrock_output_fn(response)

ModelResponse(id='chatcmpl-1aab7ca6-af66-4798-9b6d-c076ddef45f6', created=1768742695, model='bedrock-model', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content="Hello! How can I assist you today? If you have any questions or need information on a particular topic, feel free to ask. Whether it's about science, technology, history, or something else, I'm here to help.", role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None, reasoning_content=None))], usage={'prompt_tokens': 1, 'completion_tokens': 50, 'total_tokens': 51}, cache_hit=False)

In [8]:
ollama_lm = DriverLM(
    request_fn=ollama_request_fn,
    output_fn=ollama_output_fn,
    cache=True
)

ollama_lm.clear_cache()  # Clear old cache entries

native_lm = dspy.LM(
    model="ollama/llama3.2-vision:11b",
    api_base="http://localhost:11434",
    temperature=0.0
)

# Create Bedrock LM
bedrock_lm = DriverLM(
    request_fn=bedrock_request_fn,
    output_fn=bedrock_output_fn,
    temperature=0.0,
    max_tokens=2048,
    cache=True
)

bedrock_lm.clear_cache()


In [9]:
class ChatSignature(dspy.Signature):
    """Use the conversation_context to answer questions about the user."""
    conversation_context: str = dspy.InputField(desc="All previous messages in this conversation")
    user_message: str = dspy.InputField(desc="Current question")
    response: str = dspy.OutputField(desc="Answer using conversation_context")


demos = [
    dspy.Example(
        conversation_context="No previous messages",
        user_message="Hi, my name is Alice",
        response="Hello Alice! Nice to meet you."
    ).with_inputs("conversation_context", "user_message"),
    
    dspy.Example(
        conversation_context="USER: Hi, my name is Alice\nASSISTANT: Hello Alice! Nice to meet you.",
        user_message="What's my name?",
        response="Your name is Alice."  # No greeting
    ).with_inputs("conversation_context", "user_message"),
    
    # Add this demo:
    dspy.Example(
        conversation_context="USER: Hi, my name is Alice\nASSISTANT: Hello Alice!\nUSER: What's my name?\nASSISTANT: Your name is Alice.",
        user_message="How are you?",
        response="I'm doing well, thank you for asking!"  # No greeting
    ).with_inputs("conversation_context", "user_message"),
]

# Use demos in ChatAgent
class ChatAgent(dspy.Module):
    def __init__(self, demos=None):
        self.program = dspy.Predict(ChatSignature)
        if demos:
            self.program.demos = demos
        self.chat_history = []

    def forward(self, message:str):
        if len(self.chat_history) > 0:
            chat_history = "\n".join([f"{m['role'].upper()}: {m['content']}" for m in self.chat_history])
        else:
            chat_history = "No previous messages"
        response = self.program(conversation_context=chat_history, user_message=message)
        self.chat_history.append({"role": "user", "content": message})
        self.chat_history.append({"role": "assistant", "content": response.response})
        return response.response


In [10]:
# lm = ollama_lm
# lm = native_lm
lm = bedrock_lm
dspy.configure(lm=lm)
# ca = ChatAgent()
# Create agent with demos
ca = ChatAgent(demos=demos)

In [11]:
ca(message="My name is Bank. Nice to meet you.")

'Nice to meet you too, Bank!'

In [12]:
# hallucination from demos
ca(message="What's my favorite pizza topping?")

"I'm sorry, but I don't have information about your favorite pizza topping. I don't have access to personal preferences unless you've shared them with me in this conversation."

In [13]:
ca(message="For your previous answer, I love hawaiian!")

"That's great to hear! Hawaiian pizza is a popular choice. If you ever want to talk about pizza or anything else, feel free to share!"

In [14]:
ca(message="If I wanna find my new favorite pizza topping, what would you recommend?")

"There are so many delicious pizza toppings to try! Some popular ones include pepperoni, mushrooms, onions, bell peppers, olives, and bacon. If you're feeling adventurous, you might want to try something unique like pineapple, jalapeños, or even pineapple and ham. The best way to find your new favorite topping is to experiment and see what you enjoy the most. Don't be afraid to try something new!"

In [15]:
ca(message="What's my name?")

'Your name is Bank.'

In [34]:
ca.chat_history

[{'role': 'user', 'content': 'My name is Bank. Nice to meet you.'},
 {'role': 'assistant', 'content': "Hello Bank! It's nice to meet you."},
 {'role': 'user', 'content': "What's my favorite pizza topping?"},
 {'role': 'assistant',
  'content': "I'm sorry, but I don't have that information. It's best to ask you directly about your favorite pizza topping."},
 {'role': 'user', 'content': 'For your previous answer, I love hawaiian!'},
 {'role': 'assistant',
  'content': "It's great to hear that you love Hawaiian pizza! It's a delicious choice."},
 {'role': 'user',
  'content': 'If I wanna find my new favorite pizza topping, what would you recommend?'},
 {'role': 'assistant',
  'content': "That's a fun question! If you're looking to try something new, I'd recommend trying a pizza with truffle oil and mushrooms. The earthy flavor of the mushrooms combined with the richness of truffle oil can create a unique and delicious experience. However, the best topping is always personal preference, so

In [26]:
print(lm.history[-1]['messages'][-1]['content'])

[[ ## chat_history ## ]]
USER: My name is Bank. Nice to meet you.
ASSISTANT: Hello Bank! It's nice to meet you.
USER: What's my favorite pizza topping?
ASSISTANT: I don't have that information, but if you'd like to share, I'd be happy to hear about your favorite pizza topping!
USER: For your previous answer, I love hawaiian!
ASSISTANT: That sounds delicious! Hawaiian pizza with pineapple and ham is a great choice!
USER: If I wanna find my new favorite pizza topping, what would you recommend?
ASSISTANT: That's a fun question! If you're looking to try something new and delicious, I'd recommend trying a pizza with figs and prosciutto. It's a unique combination that pairs wonderfully with the creamy cheese and crispy crust. Another great option is a pizza with spicy chorizo and peppers – it's bold and flavorful!

[[ ## user_message ## ]]
What's my name?

Respond with the corresponding output fields, starting with the field `[[ ## response ## ]]`, and then ending with the marker for `[[ ## 

In [17]:
print(lm.history[-1]['messages'][0]['content'])

Your input fields are:
1. `chat_history` (str): Actual conversation history
2. `user_message` (str): Current user message
Your output fields are:
1. `response` (str): Assistant's response
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## chat_history ## ]]
{chat_history}

[[ ## user_message ## ]]
{user_message}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Respond based only on the provided chat_history. Examples are for format only.


In [18]:
print(lm.history[-1]['messages'][-1]['content'])

[[ ## chat_history ## ]]
USER: What's my favorite pizza topping?
ASSISTANT: I'm sorry, but I don't have any information about your favorite pizza topping since we haven't discussed it yet. You can tell me if you'd like to share your favorite topping.
USER: My name is Bank. Nice to meet you.
ASSISTANT: Hello Bank! It's nice to meet you. If you ever want to share your favorite pizza topping, I'm here to listen.
USER: What's my name?
ASSISTANT: Hello again! Your name is Bank. It's nice to meet you.
USER: For your previous answer, I love hawaiian!
ASSISTANT: Hello again, Bank! It's nice to meet you. Hawaiian pizza is a great choice with pineapple and ham. Enjoy!

[[ ## user_message ## ]]
If I wanna find my new favorite pizza topping, what would you recommend?

Respond with the corresponding output fields, starting with the field `[[ ## response ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


In [19]:
lm.history[-1]

{'prompt': None,
 'messages': [{'role': 'system',
   'content': "Your input fields are:\n1. `chat_history` (str): Actual conversation history\n2. `user_message` (str): Current user message\nYour output fields are:\n1. `response` (str): Assistant's response\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## chat_history ## ]]\n{chat_history}\n\n[[ ## user_message ## ]]\n{user_message}\n\n[[ ## response ## ]]\n{response}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Respond based only on the provided chat_history. Examples are for format only."},
  {'role': 'user',
   'content': '[[ ## chat_history ## ]]\nNo previous messages\n\n[[ ## user_message ## ]]\nHi, my name is Alice'},
  {'role': 'assistant',
   'content': '[[ ## response ## ]]\nHello Alice! Nice to meet you. How can I help you today?\n\n[[ ## completed ## ]]\n'},
  {'role': 'user',
   'content': "[[ ## chat_history ## ]]\nUSER: Hi, m

In [20]:
ca.chat_history

[{'role': 'user', 'content': "What's my favorite pizza topping?"},
 {'role': 'assistant',
  'content': "I'm sorry, but I don't have any information about your favorite pizza topping since we haven't discussed it yet. You can tell me if you'd like to share your favorite topping."},
 {'role': 'user', 'content': 'My name is Bank. Nice to meet you.'},
 {'role': 'assistant',
  'content': "Hello Bank! It's nice to meet you. If you ever want to share your favorite pizza topping, I'm here to listen."},
 {'role': 'user', 'content': "What's my name?"},
 {'role': 'assistant',
  'content': "Hello again! Your name is Bank. It's nice to meet you."},
 {'role': 'user', 'content': 'For your previous answer, I love hawaiian!'},
 {'role': 'assistant',
  'content': "Hello again, Bank! It's nice to meet you. Hawaiian pizza is a great choice with pineapple and ham. Enjoy!"},
 {'role': 'user',
  'content': 'If I wanna find my new favorite pizza topping, what would you recommend?'},
 {'role': 'assistant',
  '

In [21]:
lm.history

[{'prompt': None,
  'messages': [{'role': 'system',
    'content': "Your input fields are:\n1. `chat_history` (str): Actual conversation history\n2. `user_message` (str): Current user message\nYour output fields are:\n1. `response` (str): Assistant's response\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## chat_history ## ]]\n{chat_history}\n\n[[ ## user_message ## ]]\n{user_message}\n\n[[ ## response ## ]]\n{response}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Respond based only on the provided chat_history. Examples are for format only."},
   {'role': 'user',
    'content': '[[ ## chat_history ## ]]\nNo previous messages\n\n[[ ## user_message ## ]]\nHi, my name is Alice'},
   {'role': 'assistant',
    'content': '[[ ## response ## ]]\nHello Alice! Nice to meet you. How can I help you today?\n\n[[ ## completed ## ]]\n'},
   {'role': 'user',
    'content': "[[ ## chat_history ## ]]\nUS

In [19]:
from dspy.adapters.chat_adapter import ChatAdapter
import dspy

# Define your signature
class QA(dspy.Signature):
    """Answer questions accurately"""
    question: str = dspy.InputField(desc="The question")
    answer: str = dspy.OutputField(desc="The answer")

# Create adapter and generate system prompt
adapter = ChatAdapter()
# signature = QA
signature = ChatSignature

# Generate system prompt components
field_desc = adapter.format_field_description(signature)
field_structure = adapter.format_field_structure(signature)
task_desc = adapter.format_task_description(signature)

# Combine into full system prompt
system_prompt = f"{field_desc}\n{field_structure}\n{task_desc}"
print(system_prompt)

# Generate user message for specific inputs
# inputs = {"question": "What is the capital of Thailand?"}
inputs = {"chat_history": "User: Hi\nAssistant: Hello there!", "user_message": "How are you?"}
user_message = adapter.format_user_message_content(
    signature=signature,
    inputs=inputs,
    main_request=True  # Adds output format reminder
)
print("\n" + user_message)


Your input fields are:
1. `chat_history` (str): Previous chat history
2. `user_message` (str): Current user message
Your output fields are:
1. `response` (str): Assistant's resposne
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## chat_history ## ]]
{chat_history}

[[ ## user_message ## ]]
{user_message}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Response to user in a conversation

[[ ## chat_history ## ]]
User: Hi
Assistant: Hello there!

[[ ## user_message ## ]]
How are you?

Respond with the corresponding output fields, starting with the field `[[ ## response ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


In [20]:
[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_message}
]

[{'role': 'system',
  'content': "Your input fields are:\n1. `chat_history` (str): Previous chat history\n2. `user_message` (str): Current user message\nYour output fields are:\n1. `response` (str): Assistant's resposne\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## chat_history ## ]]\n{chat_history}\n\n[[ ## user_message ## ]]\n{user_message}\n\n[[ ## response ## ]]\n{response}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Response to user in a conversation"},
 {'role': 'user',
  'content': '[[ ## chat_history ## ]]\nUser: Hi\nAssistant: Hello there!\n\n[[ ## user_message ## ]]\nHow are you?\n\nRespond with the corresponding output fields, starting with the field `[[ ## response ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.'}]

In [21]:
from dspy.adapters.chat_adapter import ChatAdapter
import dspy

# Define signature
class QA(dspy.Signature):
    """Answer questions accurately"""
    question: str = dspy.InputField(desc="The question")
    answer: str = dspy.OutputField(desc="The answer")

# Create demos
demos = [
    dspy.Example(question="What is 2+2?", answer="4").with_inputs("question"),
    dspy.Example(question="What color is the sky?", answer="Blue").with_inputs("question"),
]

# Create adapter
adapter = ChatAdapter()

# Use the format() method to generate full messages including demos
messages = adapter.format(
    signature=QA,
    demos=demos,
    inputs={"question": "What is the capital of Thailand?"}
)

# Print all messages
for msg in messages:
    print(f"{msg['role'].upper()}:")
    print(msg['content'])
    print("=" * 50)


SYSTEM:
Your input fields are:
1. `question` (str): The question
Your output fields are:
1. `answer` (str): The answer
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Answer questions accurately
USER:
[[ ## question ## ]]
What is 2+2?
ASSISTANT:
[[ ## answer ## ]]
4

[[ ## completed ## ]]

USER:
[[ ## question ## ]]
What color is the sky?
ASSISTANT:
[[ ## answer ## ]]
Blue

[[ ## completed ## ]]

USER:
[[ ## question ## ]]
What is the capital of Thailand?

Respond with the corresponding output fields, starting with the field `[[ ## answer ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


In [22]:
messages

[{'role': 'system',
  'content': 'Your input fields are:\n1. `question` (str): The question\nYour output fields are:\n1. `answer` (str): The answer\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## question ## ]]\n{question}\n\n[[ ## answer ## ]]\n{answer}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Answer questions accurately'},
 {'role': 'user', 'content': '[[ ## question ## ]]\nWhat is 2+2?'},
 {'role': 'assistant',
  'content': '[[ ## answer ## ]]\n4\n\n[[ ## completed ## ]]\n'},
 {'role': 'user', 'content': '[[ ## question ## ]]\nWhat color is the sky?'},
 {'role': 'assistant',
  'content': '[[ ## answer ## ]]\nBlue\n\n[[ ## completed ## ]]\n'},
 {'role': 'user',
  'content': '[[ ## question ## ]]\nWhat is the capital of Thailand?\n\nRespond with the corresponding output fields, starting with the field `[[ ## answer ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.

In [ ]:
from dspy.adapters.chat_adapter import ChatAdapter

class QA(dspy.Signature):
    """Answer the question in concise."""
    question:str = dspy.InputField(desc="user's questioin")
    answer:str = dspy.OutputField(desc="answer to the question")

class QABot(dspy.Module):
    def __init__(self):
        self.program = dspy.Predict(QA)

    def forward(self, question):
        return self.program(question=question)

lm = native_lm
dspy.configure(lm=lm)

chat_adapter = ChatAdapter()

qa_bot = QABot()

messages = chat_adapter.format(
    signature=QA,
    demos=[],
    inputs={"question": "what's love?"}
)

messages

[{'role': 'system',
  'content': "Your input fields are:\n1. `question` (str): user's questioin\nYour output fields are:\n1. `answer` (str): answer to the question\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## question ## ]]\n{question}\n\n[[ ## answer ## ]]\n{answer}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Answer the question in concise."},
 {'role': 'user',
  'content': "[[ ## question ## ]]\nwhat's love?\n\nRespond with the corresponding output fields, starting with the field `[[ ## answer ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`."}]

In [20]:
chat_adapter.format(
    signature=QA,
    demos=[],
    inputs={"question": "what's love?"}
)

[{'role': 'system',
  'content': "Your input fields are:\n1. `question` (str): user's questioin\nYour output fields are:\n1. `answer` (str): answer to the question\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## question ## ]]\n{question}\n\n[[ ## answer ## ]]\n{answer}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Answer the question in concise."},
 {'role': 'user',
  'content': "[[ ## question ## ]]\nwhat's love?\n\nRespond with the corresponding output fields, starting with the field `[[ ## answer ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`."}]

In [17]:
qa_bot.program.signature

QA(question -> answer
    instructions='Answer the question in concise.'
    question = Field(annotation=str required=True json_schema_extra={'desc': "user's questioin", '__dspy_field_type': 'input', 'prefix': 'Question:'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'answer to the question', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)